In [78]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split

In [2]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
device = torch.device('cuda') if (USE_GPU and torch.cuda.is_available()) else torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100
print('using device:', device)

using device: cuda


In [3]:
try:
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

  FOLDERNAME = '2A/HACKATON'
  %cd /content/gdrive/My\ Drive/$FOLDERNAME
except ImportError:
  pass

Mounted at /content/gdrive
/content/gdrive/My Drive/2A/HACKATON


In [ ]:
%pwd

'/content/gdrive/My Drive/2A/HACKATON'

In [68]:
df_train = pd.read_csv('processed_waiting_times_train.csv')

In [69]:
out_features = max(df_train["WAIT_TIME_IN_2H"]) + 1
print(out_features)

32


In [70]:
y = df_train["WAIT_TIME_IN_2H"]
print(y)

x = df_train
x.drop(["WAIT_TIME_IN_2H"], axis=1, inplace=True)
print(x)

0        6
1        5
2        7
3        2
4        2
        ..
37013    2
37014    4
37015    2
37016    9
37017    4
Name: WAIT_TIME_IN_2H, Length: 37018, dtype: int64
       ADJUST_CAPACITY  DOWNTIME  CURRENT_WAIT_TIME  TIME_TO_PARADE_1  \
0            -0.660639 -0.168584          -0.240147          1.271664   
1            -0.660639 -0.168584           0.468341         -0.620346   
2            -0.527907 -0.168584           0.822585          1.271664   
3            -0.726608 -0.168584          -0.594390         -1.010215   
4            -1.033079 -0.168584          -0.594390          1.271664   
...                ...       ...                ...               ...   
37013         1.356085 -0.168584           0.468341         -0.815281   
37014        -0.527907 -0.168584          -0.594390         -0.712080   
37015         1.356085 -0.168584          -0.594390         -1.033149   
37016         1.356085 -0.168584           0.468341         -0.723547   
37017        -1.033079 -0

In [71]:
in_features = len(x.columns)
print(in_features)

84


In [112]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=0)

In [113]:
x_train = x_train.astype(float)
x_test = x_test.astype(float)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [114]:
x_train = torch.tensor(x_train.values, dtype=torch.float64, device=device)
x_test = torch.tensor(x_test.values, dtype=torch.float64, device=device)
y_train = torch.tensor(y_train.values, dtype=torch.int64, device=device)
y_test = torch.tensor(y_test.values, dtype=torch.int64, device=device)

In [115]:
#y_train = F.one_hot(y_train, out_features)
#y_test = F.one_hot(y_test, out_features)
print(y_train.shape)

torch.Size([31465])


In [127]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = nn.Sequential(
    nn.Linear(in_features, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Linear(512, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Linear(1024, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Linear(1024, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Linear(1024, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Linear(512, out_features),
)

optimizer = optim.SGD(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()
model.to(device=device)

model.train()
num_epochs = 100

for epoch in range(num_epochs):
  for inputs, targets in train_loader:
      inputs = inputs.to(device=device, dtype=torch.float)
      outputs = model(inputs)

      loss = criterion(outputs, targets)

      optimizer.zero_grad()
      loss.backward()

      optimizer.step()
  print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/100], Loss: 1.9011
Epoch [2/100], Loss: 2.0605
Epoch [3/100], Loss: 1.6330
Epoch [4/100], Loss: 1.1920
Epoch [5/100], Loss: 1.9710
Epoch [6/100], Loss: 1.4943
Epoch [7/100], Loss: 1.0544
Epoch [8/100], Loss: 1.1339
Epoch [9/100], Loss: 1.2143
Epoch [10/100], Loss: 0.6633
Epoch [11/100], Loss: 1.3209
Epoch [12/100], Loss: 1.0501
Epoch [13/100], Loss: 0.7992
Epoch [14/100], Loss: 0.8406
Epoch [15/100], Loss: 0.5765
Epoch [16/100], Loss: 0.8804
Epoch [17/100], Loss: 0.4926
Epoch [18/100], Loss: 0.7237
Epoch [19/100], Loss: 0.7034
Epoch [20/100], Loss: 1.2619
Epoch [21/100], Loss: 0.4673
Epoch [22/100], Loss: 0.4920
Epoch [23/100], Loss: 0.6902
Epoch [24/100], Loss: 0.7469
Epoch [25/100], Loss: 0.6837
Epoch [26/100], Loss: 0.2812
Epoch [27/100], Loss: 0.5655
Epoch [28/100], Loss: 0.3508
Epoch [29/100], Loss: 0.1349
Epoch [30/100], Loss: 0.6631
Epoch [31/100], Loss: 0.3938
Epoch [32/100], Loss: 0.3893
Epoch [33/100], Loss: 0.3246
Epoch [34/100], Loss: 0.4512
Epoch [35/100], Loss: 0

In [60]:
y_train.device

device(type='cuda', index=0)

In [131]:
x_test = x_test.to(device=device, dtype=torch.float)
y_hat = torch.argmax(model(x_test), dim=1)
print(y_hat)

def accuracy(y_hat, y_test):
  return torch.sum(y_hat == y_test) / len(y_hat)

print(accuracy(y_hat, y_test))

print(np.sqrt(torch.sum((5*y_hat - 5*y_test)**2 ).cpu()) / len(y_test))

tensor([7, 9, 1,  ..., 7, 7, 1], device='cuda:0')
tensor(0.5485, device='cuda:0')
tensor(0.1191, dtype=torch.float64)


#VALIDATION

In [137]:
x_val = pd.read_csv('processed_waiting_times_val.csv')
dates = x_val["DATETIME"]
entities = x_val["ENTITY_DESCRIPTION_SHORT"]
x_val.drop(["DATETIME", "ENTITY_DESCRIPTION_SHORT"], axis=1, inplace=True)

x_val = x_val.astype(float)
x_val = torch.tensor(x_val.values, dtype=torch.float64, device=device)
x_val = x_val.to(device=device, dtype=torch.float)

In [146]:
y_hat = torch.argmax(model(x_val), dim=1)

clean_y_hat = np.array((y_hat).cpu())*5
print(len(clean_y_hat))

2444


In [152]:
res_df = pd.DataFrame(list(zip(clean_y_hat, entities, dates, ["Validation" for _ in range(len(dates))])), columns=["y_pred", "ENTITY_DESCRIPTION_SHORT", "DATETIME", "KEY"])
res_df.to_csv('/content/gdrive/MyDrive/2A/HACKATON/res.csv', index=False)